<a href="https://colab.research.google.com/github/basavarajmullur/Spring-Boot-JdbcTemplate/blob/master/notebooks/quick_start_with_hugging_face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

!pip install -q transformers accelerate bitsandbytes
!pip install -q fastapi uvicorn pyngrok pillow


## Setup

To complete this tutorial, you'll need to have a runtime with [sufficient resources](https://ai.google.dev/gemma/docs/core#sizes) to run the MedGemma model.

You can try out MedGemma 4B for free in Google Colab using a T4 GPU:

1. In the upper-right of the Colab window, select **▾ (Additional connection options)**.
2. Select **Change runtime type**.
3. Under **Hardware accelerator**, select **T4 GPU**.

**Note**: To run the demo with MedGemma 27B in Google Colab, you will need a runtime with an A100 GPU.

### Get access to MedGemma

Before you get started, make sure that you have access to MedGemma models on Hugging Face:

1. If you don't already have a Hugging Face account, you can create one for free by clicking [here](https://huggingface.co/join).
2. Head over to the [MedGemma model page](https://huggingface.co/google/medgemma-1.5-4b-it) and accept the usage conditions.

### Step 1: Authenticate with Hugging Face


In [2]:
from huggingface_hub import login
login()

### Step 2: Install dependencies

In [1]:
!pip install -q \
  fastapi \
  uvicorn \
  transformers \
  accelerate \
  bitsandbytes \
  pillow==10.4.0 \
  torch torchvision \





## Step 3: Load MedGemma

In [ ]:
import torch
from transformers import AutoProcessor, AutoModelForCausalLM

MODEL_ID = "google/medgemma-4b-it"

processor = AutoProcessor.from_pretrained(MODEL_ID)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16,
    device_map="auto"
)

model.eval()
print("✅ MedGemma loaded")


The image processor of type `Gemma3ImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. 


## Step 4: Install cloudflared

In [ ]:
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64

## Step 5: MEDICAL SYSTEM PROMPT

In [ ]:
from fastapi import FastAPI, UploadFile, Form
from PIL import Image
import io
import json
from pydantic import BaseModel
from typing import Optional
import base64

class AnalyzeRequest(BaseModel):
    prompt: str
    image_base64: str
    max_tokens: int = 512

app = FastAPI(title="ClinIQ – MedGemma API")

SYSTEM_PROMPT = """
You are a clinical decision support assistant.

Rules:
- Do NOT provide diagnoses
- Use observational language only
- Explicitly state uncertainty
- Phrase findings for clinicians
- Avoid prescriptive advice

Respond ONLY with valid JSON.

JSON schema:
{
  "observations": [],
  "possible_interpretations": [],
  "uncertainty_notes": "",
  "recommend_next_steps": []
}
"""
@app.post("/debug")
def debug(req: AnalyzeRequest):
    return {
        "prompt_len": len(req.prompt),
        "image_base64_present": req.image_base64 is not None,
        "image_base64_len": len(req.image_base64),
        "max_tokens": req.max_tokens,
        "base64_prefix": req.image_base64[:30],  # sanity check
    }

@app.post("/analyze")
async def analyze(
    image: UploadFile,
    clinical_question: str = Form(...)
):
    img_bytes = await image.read()
    img = Image.open(io.BytesIO(img_bytes)).convert("RGB")

    full_prompt = f"""
Return ONLY valid JSON.

{SYSTEM_PROMPT}

Clinical question:
{clinical_question}
"""

    inputs = processor(
        images=img,
        text=full_prompt,
        return_tensors="pt"
    ).to(model.device)

    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=512,
            temperature=0.4
        )

    text = processor.decode(output_ids[0], skip_special_tokens=True)

    # enforce JSON safety
    try:
        return json.loads(text)
    except Exception:
        return {
            "error": "Model did not return valid JSON",
            "raw_output": text
        }



## Step 6: Run FastAPI server



In [ ]:
import logging
import uvicorn
from threading import Thread

# -----------------------
# Logging setup
# -----------------------
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(name)s | %(message)s"
)

logger = logging.getLogger("cliniq")

def start_api():
    logger.info("Starting FastAPI server on 127.0.0.1:8000")

    uvicorn.run(
        app,
        host="127.0.0.1",
        port=8000,
        log_level="info",
        access_log=True
    )

    logger.info("Uvicorn process exited")

Thread(target=start_api).start()


## Step 7 Expose via Cloudflare Tunnel

In [14]:
import subprocess
import re

process = subprocess.Popen(
    [
        "./cloudflared-linux-amd64",
        "tunnel",
        "--no-autoupdate",
        "--protocol", "http2",        # ❌ no QUIC
        "--url", "http://127.0.0.1:8000"
    ],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
)

for line in process.stdout:
    print(line, end="")
    if "trycloudflare.com" in line:
        print("\n🌍 COPY THIS URL ↑↑↑\n")


KeyboardInterrupt: 